In [2]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
import pandas as pd
import random
np.random.seed(10)


import sys

if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")


In [8]:


# Smooth Relu Activation Function
def swish(x):
    return x/(1+np.exp(-x))

# Smooth Relu derivative
def der_swish(x):
    return 1/(1+np.exp(-x))-(x*np.exp(-x))/((1+np.exp(-x))*(1+np.exp(-x))) 


# Smooth Relu Activation Function
def srelu(x):
    return np.log(1+np.exp(x))

# Smooth Relu derivative
def der_srelu(x):
    return (np.exp(x))/(x+np.exp(x))

# Relu Activation Function
def relu(x):
    return np.maximum(0,x)

# htan derivative
def der_relu(x):
    return np.where(x>=0,1,0)

def htan(x):
    return (np.exp(x) - np.exp(-x))/(np.exp(x) + np.exp(-x))

# htan derivative
def der_htan(x):
    return 1 - htan(x) * htan(x)

# Sigma  Activation Function
def sigma(x):
    return 1/(1+np.exp(-x))

# sigma derivative
def der_sigma(x):
    return sigma(x)*(1-sigma(x))
    
def initialize_params(layer_sizes):
    params = {}
    for i in range(1, len(layer_sizes)):
        params['W' + str(i)] = np.random.randn(layer_sizes[i], layer_sizes[i-1])*0.01
        params['B' + str(i)] = np.random.randn(layer_sizes[i],1)*0.01
    return params

def forward_propagation(X_train, params):
    layers = len(params)//2
    values = {}
    for i in range(1, layers+1):
        if i==1:
            values['Z' + str(i)] = np.dot(params['W' + str(i)], X_train) + params['B' + str(i)]
            values['A' + str(i)] = relu(values['Z' + str(i)])
        else:
            values['Z' + str(i)] = np.dot(params['W' + str(i)], values['A' + str(i-1)]) + params['B' + str(i)]
            if i==layers:
                values['A' + str(i)] = values['Z' + str(i)]
            else:
                values['A' + str(i)] = relu(values['Z' + str(i)])
    return values





def backward_propagation(params, values, X_train, Y_train):
    layers = len(params)//2
    m = len(Y_train)


    layers = len(values)//2
    Y_pred = values['A' + str(layers)]
    




    grads = {}
    for i in range(layers,0,-1):
        if i==layers:
            # dA = 1/m * (values['A' + str(i)] - Y_train)
            dA = 1/m * ((values['A' + str(i)] - Y_train))
            dZ = dA
        else:
            dA = np.dot(params['W' + str(i+1)].T, dZ)
            dZ = np.multiply(dA, der_relu(values['A' + str(i)]))
        if i==1:
            grads['W' + str(i)] = 1/m * np.dot(dZ, X_train.T)
            grads['B' + str(i)] = 1/m * np.sum(dZ, axis=1, keepdims=True)
        else:
            grads['W' + str(i)] = 1/m * np.dot(dZ,values['A' + str(i-1)].T)
            grads['B' + str(i)] = 1/m * np.sum(dZ, axis=1, keepdims=True)
    return grads

def update_params(params, grads, learning_rate):
    layers = len(params)//2
    params_updated = {}
    for i in range(1,layers+1):
        params_updated['W' + str(i)] = params['W' + str(i)] - learning_rate * grads['W' + str(i)]
        params_updated['B' + str(i)] = params['B' + str(i)] - learning_rate * grads['B' + str(i)]
    return params_updated


def model(X_train, X_test, Y_train, Y_test, layer_sizes, num_iters, learning_rate):
    params = initialize_params(layer_sizes)

    for i in range(num_iters):
        values = forward_propagation(X_train.T, params)
        tr_cost = compute_cost(values, Y_train.T)
        ts_values = forward_propagation(X_test.T, params)
        ts_cost = compute_cost(ts_values, Y_test.T)
        grads = backward_propagation(params, values,X_train.T, Y_train.T)
        params = update_params(params, grads, learning_rate)
        #print('Cost at iteration ' + str(i+1) + ' = ' + str(cost) + '\n')
        train_cost.append(tr_cost)
        test_cost.append(ts_cost)
    return params

def compute_cost(values, y):
    layers = len(values)//2
    Y_pred = values['A' + str(layers)]

    for i in range(len(y)-1):
        delta_actual=y[i+1]-y[i]
        delta_predicted=Y_pred[0][i+1]-Y_pred[0][i]

    cost=np.sum(((Y_pred - y))*((Y_pred - y)))
    return cost



def predict(X, params):
    values = forward_propagation(X.T, params)
    predictions = values['A' + str(len(values)//2)].T
    return predictions

def train_test_cost_plot(train_cost,test_cost):
    # line 1 points
    x1 = [i for i in range (1,num_iters+1)]
    y1 = train_cost[:]
    # plotting the line 1 points
    plt.plot(x1, y1, label = "Train cost")
    # line 2 points
    x2 = [i for i in range (1,num_iters+1)]
    y2 = test_cost[:]
    # plotting the line 2 points
    plt.plot(x2, y2, label = "Test cost")
    # naming the x axis
    plt.xlabel('No of iteration')
    # naming the y axis
    plt.ylabel('Value of cost')
    # giving a title to my graph
    plt.title('Two cost on same graph!')
    # show a legend on the plot
    plt.legend()
    

  

In [ ]:

if __name__ == "__main__":

    #load dataset
    df = pd.read_csv('file_pathname.csv') 

    #separate data into input and output features
    X = np.array(df[['X1','X2','X3']].values.tolist())
    # print(X.shape)
    Y = np.array(df['Y2'].values.tolist())
    
    #split data into train and test sets in 80-20 ratio
    train_data_len=int(df.shape[0]*0.8)
    X_train=X[:train_data_len]
    
    X_test=X[train_data_len:]
    Y_train=Y[:train_data_len]
    Y_test =Y[train_data_len:]
    # print(X_train,"y",Y_train)           

    train_cost=[]
    test_cost=[]


    layer_sizes = [3,5,5, 1]                                                       #set layer sizes, do not change the size of the first and last layer 
    num_iters = 100000                                                    #set number of iterations over the training set(also known as epochs in batch gradient descent context)
    learning_rate = 0.00001
    alpha = 0                                                         #set learning rate for gradient descent
    params = model(X_train, X_test, Y_train, Y_test, layer_sizes, num_iters, learning_rate)           #train the model

    train_test_cost_plot(train_cost,test_cost)
    predicted_train=predict(X_train,params)
    predicted_test=predict(X_test,params)

    
y_predicted_test=predicted_test
y_predicted_train=predicted_train
# print(predicted_test)
# print(predicted_train)